# Initalize libraries

## Import libraries

In [ ]:
import sys, os
import gc
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
from glob import glob
from tqdm.auto import tqdm
from joblib import Parallel, delayed

# Data
import xarray as xr
import h5py
import numpy as np
from sfdata import SFDataFiles, SFScanInfo, SFProcFile

# Plotting
import imageio
import matplotlib.pyplot as plt
import fabio


# scipy
from scipy.ndimage.filters import gaussian_filter
import scipy
from scipy import stats


# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
# import phrcore
import mask_lib
import helper_functions as helper
import interactive
from interactive import cimshow
import Swiss_FEL_Loading as sfl

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
    import Phase_Retrieval as PhR
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
%load_ext jupyter_black

## Experiment specific Functions

In [ ]:
BEAMTIMEID = 21625
USER = getuser()

# Number or jobs for analysis
NR_JOBS = 16

BASEFOLDER = f"/das/home/{USER}/p{BEAMTIMEID}"
mnemonics = sfl.load_mnemonics()

### Loading data

In [ ]:
# Define ids of different run numbers for image loading
run_nr = 15  # single helicity mode: image with magnetic contrast, double helicity: pos

# Find files and folders
ids, folder = sfl.list_data_filenames(run_nr, BASEFOLDER, search_key="*BSDATA.h5")

In [ ]:
# Load energy
energy = SFScanInfo(folder + "/meta/scan.json").readbacks

# Load data
data_dict = sfl.load_run(ids, mnemonics)

In [ ]:
# Create xarray
data = xr.Dataset()

# Coordinate
data["energy"] = energy

# Create variables
for key in ["Diode", "I0_intensity_Ar"]:
    data[key] = xr.DataArray(data_dict[key], dims=["idx", "channel"])

## Create coordinates
sel = (data["channel"]<10000)
data = data.isel(sel)

In [ ]:
# Sign correction
for key in ["I0_intensity_Ar"]:
    data[key] = -data[key]

In [ ]:
# Plot diode and time-of-flight signal
fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharex=True)
ax[0].plot(np.mean(data["Diode"], axis=0))
ax[0].grid()
ax[0].set_ylabel("Diode")
ax[1].plot(np.mean(data["I0_intensity_Ar"], axis=0))
ax[1].grid()
ax[1].set_ylabel("Time-of-flight I0")

In [ ]:
# Clean spectra
# Remove offset
for key in ["Diode", "I0_intensity_Ar"]:
    data[key] = data[key] - data[key][:, :1000].mean("channel")

In [ ]:
# Plot diode and time-of-flight signal
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].plot(np.mean(data["Diode"].values, axis=0))
ax[0].grid()
ax[1].plot(np.mean(data["I0_intensity_Ar"].values, axis=0))
ax[1].grid()

In [ ]:
# Get total diode and tof signal
sel = (data["channel"]) * ()

In [ ]:
# Plot diode and time-of-flight signal
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].plot(np.mean(data["Diode"], axis=0))
ax[0].grid()
ax[1].plot(np.mean(data["I0_intensity_Ar"], axis=0))
ax[1].grid()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(
    np.sum(data["Diode"], axis=0) / np.sum(data["I0_intensity_Ar"], axis=0),
    linewidth=0.5,
)
ax.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(
    np.sum(data["I0_intensity_Ar"], axis=0),
    linewidth=0.5,
)
ax.grid()

In [ ]:
fig, ax = plt

In [ ]:
data["Diode"].shape

In [ ]:
data["I0_intensity_Ar"].shape

In [ ]:
reload(sfl)

In [ ]:
sfl.load_run

In [ ]:
data